# Hybrid Monte Carlo

## Performance Analysis of Model Simulation and Payoff Calculation

In this notebook we analyse the computational effort for model state simulation and discounted payoff calculation. As implementation choices we consider pure Python/Numpy, Julia and C++ (QuantLib).

The notebook is structured as follows:

  1.  Setting up reference model and reference payoffs in Python:
        
        a) 2-Factor Quasi Gaussian model
        
        b) 10y Vanilla swap time line (similar to exposure simulation)

  2.  Run Python simulation for various number of paths scenarios

  3.  Convert model and payoffs to Julia and repeat simulations
  
  4.  (Convert model and payoffs to C++ with QuantLib and repeat simulation)


Julia (and Python) includes are relative to top-level directory. We need to make sure we find both Python and Julia files. This seem a bit tricky for Julia and we need to include the Julia modules here.

In [85]:
import os
os.chdir(r'../')
from hybmc.wrappers.JuliaSimulation import JuliaSimulation, JuliaDiscountedAt
from hybmc.wrappers.JuliaPayoff import JuliaPayoff, JuliaPayoffs

We use a couple of standard packages and QuantLib to set up the swap intrument.

In [86]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import QuantLib as ql

from timeit import default_timer as timer

### Product and Model Setup

We use a Vanilla interest rate swap as example product. The product yields a list of payoffs per observation time.

In [87]:
today     = ql.Date(5,ql.October,2020)
ql.Settings.instance().evaluationDate = today

In [88]:
from hybmc.products.Swap import Swap
discYtsH   = ql.YieldTermStructureHandle(
                 ql.FlatForward(today,0.015,ql.Actual365Fixed()))
projYtsH   = ql.YieldTermStructureHandle(
                 ql.FlatForward(today,0.020,ql.Actual365Fixed()))
index      = ql.Euribor6M(projYtsH)
startDate  = ql.Date(12,ql.October,2020)
endDate    = ql.Date(12,ql.October,2030)
calendar   = ql.TARGET()
fixedTenor = ql.Period('1y')
floatTenor = ql.Period('6m')
fixedSchedule = ql.MakeSchedule(startDate,endDate,tenor=fixedTenor,calendar=calendar)
floatSchedule = ql.MakeSchedule(startDate,endDate,tenor=floatTenor,calendar=calendar)
couponDayCount = ql.Thirty360()
notional   = 1.0
fixedRate  = 0.02
fixedLeg   = ql.FixedRateLeg(fixedSchedule,couponDayCount,[notional],[fixedRate])
floatingLeg = ql.IborLeg([notional],floatSchedule,index)
#
swap = Swap([fixedLeg,floatingLeg],[1.0,-1.0],discYtsH)

The swap product provides the basis for our payoffs. We calculae payoffs for a time line of annual observation times.

In [89]:
display(swap)
observationTimes = np.linspace(0.0,10.0,11)
timeline = swap.timeLine(observationTimes)
#display(timeline)

As reference model we use a 2-factor Quasi Gaussian model:

In [90]:
from hybmc.termstructures.YieldCurve import YieldCurve
from hybmc.models.QuasiGaussianModel import QuasiGaussianModel

yc = YieldCurve(0.02)
d = 2
times = np.array(  [ 1.0,    5.0,   10.0    ])
sigma = np.array([ [ 0.0050, 0.0060, 0.0070 ],
                   [ 0.0050, 0.0060, 0.0070 ] ])
slope = np.array([ [ 0.0100, 0.0100, 0.0100 ],
                   [ 0.0200, 0.0200, 0.0200 ] ])
curve = np.array([ [ 0.0000, 0.0000, 0.0000 ],
                   [ 0.0000, 0.0000, 0.0000 ] ])
delta = np.array(  [ 1.0,  20.0 ])
chi   = np.array(  [ 0.01, 0.15 ])
Gamma = np.identity(2)

model = QuasiGaussianModel(yc,d,times,sigma,slope,curve,delta,chi,Gamma)

We also specify the scenarios in terms of number of Monte Carlo paths. Also we set some further parameters for simulation.

In [91]:
# nPathScenarios = np.array([ 2**k for k in [7, 8, 9, 10, 11 ] ])
nPathScenarios = np.array([ 2**k for k in [7, 8, 9, 10, 11, 12, 13, 14] ])
# simTimes = np.array([0.0, 10.0, 11])
simTimes = observationTimes
seed = 314159
timeInterpolation = True

### Python simulation



A time line is a map of observation times to list of payoffs. We can calculate the scenarios as follows:

In [92]:
def scenariosPython(timeLine, sim):
    return np.array([ 
        [ sum([ payoff.discountedAt(path) for payoff in timeLine[t] ]) for t in timeLine ]
        for path in sim.paths() ])

This allows now to script the Python run.

In [93]:
from hybmc.simulations.McSimulation import McSimulation

def pythonRun():
    results = {}
    for nPaths in nPathScenarios:
        print('nPaths: %6d' % nPaths, end='', flush=True )
        start = timer()
        sim = McSimulation(model,simTimes,nPaths,seed,timeInterpolation, False)
        stop = timer()
        sim_time = stop - start
        print('  sim_time: %6.2f' % sim_time, end='', flush=True )
        start = timer()
        scenarios = scenariosPython(timeline,sim)
        stop = timer()
        pay_time = stop - start
        print('  pay_time: %6.2f' % pay_time, end='\n', flush=True )
        npvs = np.average(scenarios,axis=0)
        #print(npvs.shape)
        results[nPaths] = {
            'sim_time' : sim_time,
            'pay_time' : pay_time,
            'scen_npv' : npvs
        }
    return results

pythonResults = pythonRun()

nPaths:    128  sim_time:   0.21  pay_time:   1.47
nPaths:    256  sim_time:   0.37  pay_time:   3.01
nPaths:    512  sim_time:   0.82  pay_time:   6.00
nPaths:   1024  sim_time:   1.58  pay_time:  11.57
nPaths:   2048  sim_time:   3.11  pay_time:  23.12
nPaths:   4096  sim_time:   6.30  pay_time:  50.80
nPaths:   8192  sim_time:  13.54  pay_time: 100.96
nPaths:  16384  sim_time:  29.87  pay_time: 205.95


### Julia Simulation

We need a reference Python simulation from which we can build the Julia simulation

In [94]:
from hybmc.simulations.McSimulation import McSimulation
sim = McSimulation(model,simTimes,1,seed,timeInterpolation, False)

In [95]:
jTimeline = { t : JuliaPayoffs(timeline[t]) for t in timeline }

In [96]:
def scenariosJulia(jTimeline, jSim):
    return np.array([ np.sum(JuliaDiscountedAt(jSim,jTimeline[t]),axis=0)
        for t in jTimeline ])

Now we can script the Julia run.

In [97]:
def juliaRun():
    results = {}
    for nPaths in nPathScenarios:
        print('nPaths: %6d' % nPaths, end='', flush=True )
        start = timer()
        jSim = JuliaSimulation(sim,simulate=True,nPaths=int(nPaths))
        stop = timer()
        sim_time = stop - start
        print('  sim_time: %6.2f' % sim_time, end='', flush=True )
        start = timer()
        scenarios = scenariosJulia(jTimeline, jSim)
        stop = timer()
        pay_time = stop - start
        print('  pay_time: %6.2f' % pay_time, end='\n', flush=True )
        npvs = np.average(scenarios,axis=1)
        #print(npvs.shape)
        results[nPaths] = {
            'sim_time' : sim_time,
            'pay_time' : pay_time,
            'scen_npv' : npvs
        }
    return results

juliaResults = juliaRun()

nPaths:    128  sim_time:   0.08  pay_time:   1.87
nPaths:    256  sim_time:   0.04  pay_time:   2.02
nPaths:    512  sim_time:   0.06  pay_time:   2.12
nPaths:   1024  sim_time:   0.11  pay_time:   2.49
nPaths:   2048  sim_time:   0.26  pay_time:   3.50
nPaths:   4096  sim_time:   0.44  pay_time:   5.30
nPaths:   8192  sim_time:   0.96  pay_time:   7.82
nPaths:  16384  sim_time:   1.99  pay_time:  14.63


### C++ (QuantLib) Simulation

*
The following code requires a custom QuantLib library, see https://github.com/sschlenkrich/QuantLib.
If you do not have QuantLib or if it does not work then disable via the flag 'useQuantLib'.
*

In [98]:
useQuantLib = True

In [99]:
if useQuantLib:
    from hybmc.wrappers.QuantLibSimulation import QuantLibSimulation, QuantLibDiscountedAt
    from hybmc.wrappers.QuantLibPayoffs import QuantLibPayoff, QuantLibPayoffs

In [100]:
if useQuantLib:
    qTimeline = { t : QuantLibPayoffs(timeline[t]) for t in timeline }

In [101]:
def scenariosQuantLib(qTimeline, qSim):
    return np.array([ np.sum(QuantLibDiscountedAt(qSim,qTimeline[t]),axis=0)
        for t in qTimeline ])

In [102]:
def quantLibRun():
    results = {}
    for nPaths in nPathScenarios:
        print('nPaths: %6d' % nPaths, end='', flush=True )
        start = timer()
        qSim = QuantLibSimulation(sim,nPaths=int(nPaths))
        stop = timer()
        sim_time = stop - start
        print('  sim_time: %6.2f' % sim_time, end='', flush=True )
        start = timer()
        scenarios = scenariosQuantLib(qTimeline, qSim)
        stop = timer()
        pay_time = stop - start
        print('  pay_time: %6.2f' % pay_time, end='\n', flush=True )
        npvs = np.average(scenarios,axis=1)
        #print(npvs.shape)
        results[nPaths] = {
            'sim_time' : sim_time,
            'pay_time' : pay_time,
            'scen_npv' : npvs
        }
    return results

if useQuantLib:
    quantLibResults = quantLibRun()

nPaths:    128  sim_time:   0.01  pay_time:   0.03
nPaths:    256  sim_time:   0.01  pay_time:   0.06
nPaths:    512  sim_time:   0.02  pay_time:   0.11
nPaths:   1024  sim_time:   0.05  pay_time:   0.19
nPaths:   2048  sim_time:   0.07  pay_time:   0.37
nPaths:   4096  sim_time:   0.14  pay_time:   0.71
nPaths:   8192  sim_time:   0.26  pay_time:   1.47
nPaths:  16384  sim_time:   0.55  pay_time:   3.15


### Comparison of Python and Julia and C++ (QuantLib)

In [103]:
table = pd.DataFrame(nPathScenarios, columns=['nPaths'])
sim_time_cols = []
pay_time_cols = []

try:
    table['Python_sim_time'] = [ pythonResults[k]['sim_time'] for k in pythonResults ]
    table['Python_pay_time'] = [ pythonResults[k]['pay_time'] for k in pythonResults ]
    sim_time_cols += ['Python_sim_time']
    pay_time_cols += ['Python_pay_time']
except NameError:
    print('Warning: No Python results available.')

try:
    table['Julia_sim_time'] = [ juliaResults[k]['sim_time'] for k in juliaResults ]
    table['Julia_pay_time'] = [ juliaResults[k]['pay_time'] for k in juliaResults ]
    sim_time_cols += ['Julia_sim_time']
    pay_time_cols += ['Julia_pay_time']
except NameError:
    print('Warning: No Julia results available.')

try:
    table['QuantLib_sim_time'] = [ quantLibResults[k]['sim_time'] for k in quantLibResults ]
    table['QuantLib_pay_time'] = [ quantLibResults[k]['pay_time'] for k in quantLibResults ]
    sim_time_cols += ['QuantLib_sim_time']
    pay_time_cols += ['QuantLib_pay_time']
except NameError:
    print('Warning: No QuantLib results available.')

In [104]:
fig = px.line(table, x='nPaths', y = sim_time_cols, labels={ 'value' : 'run time (s)' } ,log_x=True, log_y=True)
fig.show()

In [105]:
fig = px.line(table, x='nPaths', y = pay_time_cols, labels={ 'value' : 'run time (s)' }, log_x=True, log_y=True)
fig.show()